## Conda env create

We create a conda environment for the runtime of our explainer from the `./artifacts/income_explainer/conda.yaml`
This only needs to be done once.

In [1]:
!conda env create --name tempo-explainer-example --file ./artifacts/income_explainer/conda.yaml

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.12
  latest version: 4.9.2

Please update conda by running

    $ conda update -n base -c defaults conda


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Ran pip subprocess with arguments:
['/home/clive/anaconda3/envs/tempo-explainer-example/bin/python', '-m', 'pip', 'install', '-U', '-r', '/home/clive/work/mlops/fork-tempo/notebooks/artifacts/income_explainer/condaenv.9j0v4jil.requirements.txt']
Pip subprocess output:
  Using cached mlops_tempo-0.1.0.dev4-py3-none-any.whl (38 kB)
  Using cached alibi-0.5.6-py3-none-any.whl (232 kB)
  Using cached dill-0.3.3-py2.py3-none-any.whl (81 kB)
  Using cached mlserver-0.3.1.dev5-py3-none-any.whl (45 kB)
  Using cached mlserver_tempo-0.3.1.dev5-py3-none-any.whl (6.4 kB)
  Using cached protobuf-3.15.5-cp37-cp37m-manylinux1_x86_64.whl (1.0 MB)
  Using cached grpcio-1.36.1-cp37-cp37m-manylinux2014_x86_64.whl (4.

## Setup and download Data

In [2]:
from tempo.serve.metadata import ModelFramework
from tempo.serve.model import Model
from tempo.seldon.docker import SeldonDockerRuntime
from tempo.kfserving.protocol import KFServingV2Protocol
from tempo.serve.utils import pipeline, predictmethod
from tempo.seldon.k8s import SeldonKubernetesRuntime
from tempo.serve.metadata import ModelFramework, KubernetesOptions
from alibi.utils.wrappers import ArgmaxTransformer
from typing import Any

import numpy as np
import os 
import pprint
import dill
import json

EXPLAINER_FOLDER = os.getcwd()+"/artifacts/income_explainer"
MODEL_FOLDER = os.getcwd()+"/artifacts/income_model"

In [3]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from alibi.explainers import AnchorTabular
from alibi.datasets import fetch_adult

adult = fetch_adult()
data = adult.data
target = adult.target
feature_names = adult.feature_names
category_map = adult.category_map

In [4]:
np.random.seed(0)
data_perm = np.random.permutation(np.c_[data, target])
data = data_perm[:,:-1]
target = data_perm[:,-1]
idx = 30000
X_train,Y_train = data[:idx,:], target[:idx]
X_test, Y_test = data[idx+1:,:], target[idx+1:]

## Build SKLearn Model and Alibi Anchors Tabular Explainer

In [5]:
ordinal_features = [x for x in range(len(feature_names)) if x not in list(category_map.keys())]
ordinal_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                                      ('scaler', StandardScaler())])
categorical_features = list(category_map.keys())
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                                          ('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('num', ordinal_transformer, ordinal_features),
                                               ('cat', categorical_transformer, categorical_features)])
clf = RandomForestClassifier(n_estimators=50)
model=Pipeline(steps=[("preprocess",preprocessor),("model",clf)])
model.fit(X_train,Y_train)


print('Train accuracy: ', accuracy_score(Y_train, model.predict(X_train)))
print('Test accuracy: ', accuracy_score(Y_test, model.predict(X_test)))

Train accuracy:  0.9656333333333333
Test accuracy:  0.854296875


In [6]:
from alibi.explainers import AnchorTabular
predict_fn = lambda x: model.predict(x)
explainer = AnchorTabular(predict_fn, feature_names, categorical_names=category_map, seed=1)
explainer.fit(X_train, disc_perc=[25, 50, 75])

AnchorTabular(meta={
  'name': 'AnchorTabular',
  'type': ['blackbox'],
  'explanations': ['local'],
  'params': {'disc_perc': [25, 50, 75], 'seed': 1}}
)

In [7]:
explanation = explainer.explain(X_test[0], threshold=0.95)
print('Anchor: %s' % (' AND '.join(explanation.anchor)))
print('Precision: %.2f' % explanation.precision)
print('Coverage: %.2f' % explanation.coverage)

Anchor: Marital Status = Separated AND Sex = Female AND Capital Gain <= 0.00
Precision: 0.98
Coverage: 0.10


In [34]:
from joblib import dump
dump(model, MODEL_FOLDER+"/model.joblib") 
with open(EXPLAINER_FOLDER+"/explainer.dill", 'wb') as f:
    dill.dump(explainer,f)

## Create Tempo Artifacts

In [9]:
k8s_options = KubernetesOptions(namespace="production")
k8s_runtime = SeldonKubernetesRuntime(k8s_options=k8s_options)

sklearn_model = Model(
        name="income-sklearn",
        runtime=SeldonDockerRuntime(),
        platform=ModelFramework.SKLearn,
        local_folder=MODEL_FOLDER,
        uri="gs://seldon-models/test/income/model"
)


In [14]:


@pipeline(name="income-explainer",
          runtime=SeldonDockerRuntime(protocol=KFServingV2Protocol()),
          uri="gs://seldon-models/test/income/explainer",
          local_folder=EXPLAINER_FOLDER,
          conda_env="tempo-explainer-example",
          models=[sklearn_model])
class ExplainerPipeline(object):

    def __init__(self):
        if "MLSERVER_MODELS_DIR" in os.environ:
            models_folder = ""
        else:
            models_folder = EXPLAINER_FOLDER
        with open(models_folder+"/explainer.dill", "rb") as f:
            self.explainer = dill.load(f)
        self.ran_init = True
        
    def update_predict_fn(self, x):
        if np.argmax(sklearn_model(x).shape) == 0:
            self.explainer.predictor = sklearn_model
            self.explainer.samplers[0].predictor = sklearn_model
        else:
            self.explainer.predictor = ArgmaxTransformer(sklearn_model)
            self.explainer.samplers[0].predictor = ArgmaxTransformer(sklearn_model)

    @predictmethod
    def explain(self, payload: np.ndarray, parameters: dict) -> str:
        print("Explain called with ", parameters)
        if not self.ran_init:
            print("Loading explainer")
            self.__init__()
        self.update_predict_fn(payload)
        explanation = self.explainer.explain(payload, **parameters)
        return explanation.to_json()

### Deploy model to Docker and test

In [25]:
sklearn_model.deploy()
sklearn_model.wait_ready()

True

In [26]:
sklearn_model(X_test[0:1])

array([[0.9, 0.1]])

### Create explainer and test against model

In [27]:
p = ExplainerPipeline() 

In [23]:
p.save(save_env=True)

Packing environment at '/home/clive/anaconda3/envs/tempo-explainer-example' to '/home/clive/work/mlops/fork-tempo/notebooks/artifacts/income_explainer/environment.tar.gz'
[########################################] | 100% Completed |  1min  2.8s


In [28]:
r = json.loads(p.explain(X_test[0:1], {"threshold":0.95}))
print(r["data"]["anchor"])

Explain called with  {'threshold': 0.95}
['Marital Status = Separated', 'Sex = Female']


### Deploy explainer to docker

In [29]:
p.deploy()
p.wait_ready()

True

In [30]:
r = json.loads(p.remote(payload=X_test[0:1], parameters={"threshold":0.99}))
print(r["data"]["anchor"])

Create v2 from any
['Marital Status = Separated', 'Relationship = Unmarried']


In [31]:
p.undeploy()

### Deploy to production on Kubernetes

In [32]:
k8s_options = KubernetesOptions(namespace="production")
k8s_runtime = SeldonKubernetesRuntime(k8s_options=k8s_options)
k8s_runtime_v2 = SeldonKubernetesRuntime(k8s_options=k8s_options, protocol=KFServingV2Protocol())

sklearn_model.set_runtime(k8s_runtime)
p.set_runtime(k8s_runtime_v2)

In [35]:
p.save(save_env=False)

Upload artifacts. **This step may take some time**

In [36]:
sklearn_model.upload()
p.upload()

In [37]:
p.deploy()
p.wait_ready()

True

In [39]:
r = json.loads(p.remote(payload=X_test[0:1], parameters={"threshold":0.95}))
print(r["data"]["anchor"])

Create v2 from any
['Relationship = Unmarried', 'Sex = Female']


In [40]:
p.undeploy()

## Prepare for Gitops

In [41]:
yaml = p.to_k8s_yaml()
print (eval(pprint.pformat(yaml)))

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: income-explainer
  namespace: production
spec:
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - args: []
          env:
          - name: MLSERVER_HTTP_PORT
            value: '9000'
          - name: MLSERVER_GRPC_PORT
            value: '9500'
          - name: MLSERVER_MODEL_IMPLEMENTATION
            value: mlserver_tempo.TempoModel
          - name: MLSERVER_MODEL_NAME
            value: income-explainer
          - name: MLSERVER_MODEL_URI
            value: /mnt/models
          image: seldonio/mlserver:0.3.1.dev5
          name: income-explainer
    graph:
      implementation: TRITON_SERVER
      modelUri: gs://seldon-models/test/income/explainer
      name: income-explainer
      serviceAccountName: tempo-pipeline
      type: MODEL
    name: default
    replicas: 1
  protocol: kfserving

---
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  na